In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from pathlib import Path

from core.models.model_factory import create_model
from core.data.dataset import EmotionDataset
from core.training.trainer import train_model

if __name__ == '__main__':
    # 설정값 정의
    DATA_DIR = Path("./datasets/korean_emotion_complex_vision_1_percent")
    MODEL_NAME = 'resnet18'
    NUM_CLASSES = 7  # 데이터셋의 클래스 수에 맞게 조정. ['기쁨', '당황', '분노', '불안', '상처', '슬픔', '중립']
    BATCH_SIZE = 16
    NUM_EPOCHS = 10
    LEARNING_RATE = 0.001

    # 데이터 준비
    dataset = EmotionDataset(data_dir=DATA_DIR)
    # 실제 클래스 수를 데이터셋에서 가져와 업데이트
    NUM_CLASSES = len(dataset.classes)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    print("데이터 준비 완료!")
    print(f"훈련 데이터셋 크기: {len(dataset)}")
    print(f"클래스 수: {NUM_CLASSES} -> {dataset.classes}")

    # 모델, 손실 함수, 옵티마이저 준비
    model = create_model(model_name=MODEL_NAME, num_classes=NUM_CLASSES)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    print(f"'{MODEL_NAME}' 모델, 손실 함수, 옵티마이저 준비 완료!")

    # 모델 훈련 시작
    print("\n모델 훈련을 시작합니다...")
    trained_model = train_model(model, dataloader, criterion, optimizer, num_epochs=NUM_EPOCHS)
    
    # 훈련된 모델 저장 (옵션)
    # torch.save(trained_model.state_dict(), f'{MODEL_NAME}_trained.pth')
    # print("훈련된 모델 가중치가 저장되었습니다.")

데이터 준비 완료!
훈련 데이터셋 크기: 2210
클래스 수: 7 -> ['기쁨', '당황', '분노', '불안', '상처', '슬픔', '중립']
'resnet18' 모델, 손실 함수, 옵티마이저 준비 완료!

모델 훈련을 시작합니다...
Epoch 1/10
----------
  Batch 10/139 Loss: 2.2997
  Batch 20/139 Loss: 1.8917
  Batch 30/139 Loss: 1.9244
  Batch 40/139 Loss: 1.7704
  Batch 50/139 Loss: 1.8724
  Batch 60/139 Loss: 2.2426
  Batch 70/139 Loss: 1.9314
  Batch 80/139 Loss: 1.7778
  Batch 90/139 Loss: 1.8801
  Batch 100/139 Loss: 2.2224
  Batch 110/139 Loss: 1.9506
  Batch 120/139 Loss: 2.0037
  Batch 130/139 Loss: 2.0805
Train Loss: 1.9858 Acc: 0.2054

Epoch 2/10
----------
  Batch 10/139 Loss: 1.7440
  Batch 20/139 Loss: 2.0319
  Batch 30/139 Loss: 1.7554
  Batch 40/139 Loss: 2.0765
  Batch 50/139 Loss: 1.9001
  Batch 60/139 Loss: 1.8847
  Batch 70/139 Loss: 1.8932
  Batch 80/139 Loss: 2.0150
  Batch 90/139 Loss: 1.6017
  Batch 100/139 Loss: 1.8648
  Batch 110/139 Loss: 1.8648
  Batch 120/139 Loss: 1.6670
  Batch 130/139 Loss: 1.8681
Train Loss: 1.8417 Acc: 0.2566

Epoch 3/10
----------

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from pathlib import Path

from core.models.model_factory import create_model
from core.data.dataset import EmotionDataset
from core.training.trainer import train_model

if __name__ == '__main__':
    # 설정값 정의
    # 장치 설정: 사용 가능한 경우 GPU(cuda)를, 그렇지 않으면 CPU를 사용
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {DEVICE}")
    
    DATA_DIR = Path("./datasets/korean_emotion_complex_vision_1_percent_verified_processed")
    MODEL_NAME = 'resnet18'
    NUM_CLASSES = 7  # 데이터셋의 클래스 수에 맞게 조정. ['기쁨', '당황', '분노', '불안', '상처', '슬픔', '중립']
    BATCH_SIZE = 16
    NUM_EPOCHS = 10
    LEARNING_RATE = 0.001

    # 훈련용과 검증용 데이터셋을 각각 생성.
    train_dataset = EmotionDataset(data_dir=DATA_DIR / "train")
    val_dataset = EmotionDataset(data_dir=DATA_DIR / "val")

    # 데이터로더를 각각 생성. (검증용은 섞을 필요가 없음)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    NUM_CLASSES = len(train_dataset.classes)
    
    print("데이터 준비 완료!")
    print(f"훈련 데이터셋 크기: {len(train_dataset)}")
    print(f"클래스 수: {NUM_CLASSES} -> {train_dataset.classes}")

    # 모델, 손실 함수, 옵티마이저 준비
    model = create_model(model_name=MODEL_NAME, num_classes=NUM_CLASSES)
    # 모델을 지정된 장치로 이동
    model.to(DEVICE)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    print(f"'{MODEL_NAME}' 모델, 손실 함수, 옵티마이저 준비 완료!")

    # 모델 훈련 시작
    print("\n모델 훈련을 시작합니다...")
    trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, DEVICE, num_epochs=NUM_EPOCHS)
    
    # 훈련된 모델 저장 (옵션)
    # torch.save(trained_model.state_dict(), f'{MODEL_NAME}_trained.pth')
    # print("훈련된 모델 가중치가 저장되었습니다.")

Using device: cpu
데이터 준비 완료!
훈련 데이터셋 크기: 1781
클래스 수: 7 -> ['기쁨', '당황', '분노', '불안', '상처', '슬픔', '중립']
'resnet18' 모델, 손실 함수, 옵티마이저 준비 완료!

모델 훈련을 시작합니다...
Epoch 1/10
----------
Train Loss: 1.6376 Acc: 0.3751
Val Loss: 1.7085 Acc: 0.3849

Epoch 2/10
----------
Train Loss: 1.3697 Acc: 0.4818
Val Loss: 1.4288 Acc: 0.4430

Epoch 3/10
----------
Train Loss: 1.2256 Acc: 0.5441
Val Loss: 1.4799 Acc: 0.4645

Epoch 4/10
----------
Train Loss: 1.0943 Acc: 0.6070
Val Loss: 1.4241 Acc: 0.4602

Epoch 5/10
----------
Train Loss: 0.9580 Acc: 0.6536
Val Loss: 2.0359 Acc: 0.4258

Epoch 6/10
----------
Train Loss: 0.8120 Acc: 0.7041
Val Loss: 2.3757 Acc: 0.3871

Epoch 7/10
----------
Train Loss: 0.7360 Acc: 0.7367
Val Loss: 1.6722 Acc: 0.4860

Epoch 8/10
----------
Train Loss: 0.4806 Acc: 0.8293
Val Loss: 2.2134 Acc: 0.4624

Epoch 9/10
----------
Train Loss: 0.4149 Acc: 0.8518
Val Loss: 1.9103 Acc: 0.4796

Epoch 10/10
----------
Train Loss: 0.3017 Acc: 0.9051
Val Loss: 2.1917 Acc: 0.5032

Training complet

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms

from pathlib import Path

from core.models.model_factory import create_model
from core.data.dataset import EmotionDataset
from core.training.trainer import train_model

if __name__ == '__main__':
    # 설정값 정의
    # 장치 설정: 사용 가능한 경우 GPU(cuda)를, 그렇지 않으면 CPU를 사용
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {DEVICE}")
    
    DATA_DIR = Path("./datasets/korean_emotion_complex_vision_1_percent_verified_processed")
    MODEL_NAME = 'resnet18'
    NUM_CLASSES = 7  # 데이터셋의 클래스 수에 맞게 조정. ['기쁨', '당황', '분노', '불안', '상처', '슬픔', '중립']
    BATCH_SIZE = 16
    NUM_EPOCHS = 10
    LEARNING_RATE = 0.001
    
    # 데이터 증강을 포함한 훈련용 Transform 정의
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),  # 50% 확률로 좌우 반전
        transforms.RandomRotation(15),           # -15도 ~ 15도 사이로 랜덤 회전
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), # 밝기, 대비, 채도 조절
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    
    # 증강이 없는 검증/테스트용 Transform 정의
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # 훈련용과 검증용 데이터셋을 각각 생성.
    train_dataset = EmotionDataset(data_dir=DATA_DIR / "train", transform=train_transform)
    val_dataset = EmotionDataset(data_dir=DATA_DIR / "val", transform=val_transform)

    # 데이터로더를 각각 생성. (검증용은 섞을 필요가 없음)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    NUM_CLASSES = len(train_dataset.classes)
    
    print("데이터 준비 완료!")
    print(f"훈련 데이터셋 크기: {len(train_dataset)}")
    print(f"클래스 수: {NUM_CLASSES} -> {train_dataset.classes}")

    # 모델, 손실 함수, 옵티마이저 준비
    model = create_model(model_name=MODEL_NAME, num_classes=NUM_CLASSES)
    # 모델을 지정된 장치로 이동
    model.to(DEVICE)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    print(f"'{MODEL_NAME}' 모델, 손실 함수, 옵티마이저 준비 완료!")

    # 모델 훈련 시작
    print("\n모델 훈련을 시작합니다...")
    trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, DEVICE, num_epochs=NUM_EPOCHS)
    
    # 훈련된 모델 저장 (옵션)
    # torch.save(trained_model.state_dict(), f'{MODEL_NAME}_trained.pth')
    # print("훈련된 모델 가중치가 저장되었습니다.")

Using device: cuda
데이터 준비 완료!
훈련 데이터셋 크기: 1781
클래스 수: 7 -> ['기쁨', '당황', '분노', '불안', '상처', '슬픔', '중립']
'resnet18' 모델, 손실 함수, 옵티마이저 준비 완료!

모델 훈련을 시작합니다...
Epoch 1/10
----------
Train Loss: 1.6543 Acc: 0.3678
Val Loss: 1.5184 Acc: 0.4258

Epoch 2/10
----------
Train Loss: 1.4312 Acc: 0.4576
Val Loss: 1.7822 Acc: 0.4409

Epoch 3/10
----------
Train Loss: 1.3846 Acc: 0.4874
Val Loss: 1.6696 Acc: 0.4409

Epoch 4/10
----------
Train Loss: 1.3004 Acc: 0.5126
Val Loss: 1.3269 Acc: 0.5011

Epoch 5/10
----------
Train Loss: 1.2705 Acc: 0.5222
Val Loss: 1.3473 Acc: 0.5118

Epoch 6/10
----------
Train Loss: 1.2593 Acc: 0.5334
Val Loss: 1.4601 Acc: 0.4903

Epoch 7/10
----------
Train Loss: 1.2002 Acc: 0.5503
Val Loss: 1.5093 Acc: 0.4667

Epoch 8/10
----------
Train Loss: 1.1056 Acc: 0.6036
Val Loss: 1.5257 Acc: 0.4989

Epoch 9/10
----------
Train Loss: 1.1043 Acc: 0.5856
Val Loss: 1.3051 Acc: 0.5656

Epoch 10/10
----------
Train Loss: 1.1080 Acc: 0.5794
Val Loss: 1.2696 Acc: 0.5677

Training comple

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms

from pathlib import Path

from core.models.model_factory import create_model
from core.data.dataset import EmotionDataset
from core.training.trainer import train_model

if __name__ == '__main__':
    # 설정값 정의
    # 장치 설정: 사용 가능한 경우 GPU(cuda)를, 그렇지 않으면 CPU를 사용
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {DEVICE}")
    
    DATA_DIR = Path("./datasets/korean_emotion_complex_vision_1_percent_verified_processed")
    MODEL_NAME = 'resnet18'
    NUM_CLASSES = 7  # 데이터셋의 클래스 수에 맞게 조정해야 합니다. ['기쁨', '당황', '분노', '불안', '상처', '슬픔', '중립']
    BATCH_SIZE = 16
    NUM_EPOCHS = 10
    LEARNING_RATE = 0.001
    
    # 데이터 증강을 포함한 훈련용 Transform 정의
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),  # 50% 확률로 좌우 반전
        transforms.RandomRotation(15),           # -15도 ~ 15도 사이로 랜덤 회전
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), # 밝기, 대비, 채도 조절
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    
    # 증강이 없는 검증/테스트용 Transform 정의
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # 각 데이터셋에 맞는 Transform 적용
    train_dataset = EmotionDataset(data_dir=DATA_DIR / "train", transform=train_transform)
    val_dataset = EmotionDataset(data_dir=DATA_DIR / "val", transform=val_transform)

    # 훈련용과 검증용 데이터셋을 각각 생성.
    train_dataset = EmotionDataset(data_dir=DATA_DIR / "train", transform=train_transform)
    val_dataset = EmotionDataset(data_dir=DATA_DIR / "val", transform=val_transform)

    # 데이터로더를 각각 생성. (검증용은 섞을 필요가 없음)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    NUM_CLASSES = len(train_dataset.classes)
    
    print("데이터 준비 완료!")
    print(f"훈련 데이터셋 크기: {len(train_dataset)}")
    print(f"클래스 수: {NUM_CLASSES} -> {train_dataset.classes}")

    # 모델, 손실 함수, 옵티마이저 준비
    model = create_model(model_name=MODEL_NAME, num_classes=NUM_CLASSES)
    # 모델을 지정된 장치로 이동
    model.to(DEVICE)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4) 

    print(f"'{MODEL_NAME}' 모델, 손실 함수, 옵티마이저 준비 완료!")

    # 모델 훈련 시작
    print("\n모델 훈련을 시작합니다...")
    trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, DEVICE, num_epochs=NUM_EPOCHS)
    
    # 훈련된 모델 저장 (옵션)
    # torch.save(trained_model.state_dict(), f'{MODEL_NAME}_trained.pth')
    # print("훈련된 모델 가중치가 저장되었습니다.")

Using device: cuda
데이터 준비 완료!
훈련 데이터셋 크기: 1781
클래스 수: 7 -> ['기쁨', '당황', '분노', '불안', '상처', '슬픔', '중립']
'resnet18' 모델, 손실 함수, 옵티마이저 준비 완료!

모델 훈련을 시작합니다...
Epoch 1/10
----------
Train Loss: 1.7537 Acc: 0.3217
Val Loss: 1.7876 Acc: 0.3419

Epoch 2/10
----------
Train Loss: 1.4793 Acc: 0.4301
Val Loss: 1.5791 Acc: 0.4172

Epoch 3/10
----------
Train Loss: 1.3821 Acc: 0.4739
Val Loss: 1.7380 Acc: 0.4301

Epoch 4/10
----------
Train Loss: 1.3518 Acc: 0.4941
Val Loss: 1.6685 Acc: 0.3591

Epoch 5/10
----------
Train Loss: 1.2540 Acc: 0.5255
Val Loss: 1.5061 Acc: 0.4301

Epoch 6/10
----------
Train Loss: 1.2534 Acc: 0.5182
Val Loss: 1.2826 Acc: 0.5247

Epoch 7/10
----------
Train Loss: 1.1835 Acc: 0.5609
Val Loss: 1.3643 Acc: 0.5161

Epoch 8/10
----------
Train Loss: 1.1819 Acc: 0.5615
Val Loss: 1.3488 Acc: 0.5183

Epoch 9/10
----------
Train Loss: 1.1253 Acc: 0.5705
Val Loss: 1.3381 Acc: 0.4968

Epoch 10/10
----------
Train Loss: 1.1238 Acc: 0.5879
Val Loss: 1.5432 Acc: 0.4731

Training comple

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms

from pathlib import Path

from core.models.model_factory import create_model
from core.data.dataset import EmotionDataset
from core.training.trainer import train_model

from torch.optim.lr_scheduler import StepLR

if __name__ == '__main__':
    # 설정값 정의
    # 장치 설정: 사용 가능한 경우 GPU(cuda)를, 그렇지 않으면 CPU를 사용
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {DEVICE}")
    
    DATA_DIR = Path("./datasets/korean_emotion_complex_vision_5_percent_verified_processed")
    # 사용하고자 하는 모델 하나만 남기고 다른 MODEL_NAME 앞에 # 붙여서 주석처리
    #MODEL_NAME = 'resnet18'             #철원
    #MODEL_NAME = 'resnet50' 
    #MODEL_NAME = 'mobilenet_v3_small'  #승현님
    #MODEL_NAME = 'shufflenet_v2'       #철원
    MODEL_NAME = 'efficientnet_v2_s'   #규진님
    #MODEL_NAME = 'squeezenet'          #승희님
    
    NUM_CLASSES = 7  # 데이터셋의 클래스 수에 맞게 조정해야 합니다. ['기쁨', '당황', '분노', '불안', '상처', '슬픔', '중립']
    BATCH_SIZE = 16
    LEARNING_RATE = 0.001
    NUM_EPOCHS = 100
    EARLY_STOPPING_PATIENCE = 10 # 10번 연속 성능 개선이 없으면 조기 종료
    
    # 데이터 증강을 포함한 훈련용 Transform 정의
    train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),  # 50% 확률로 좌우 반전
        transforms.RandomRotation(15),           # -15도 ~ 15도 사이로 랜덤 회전
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), # 밝기, 대비, 채도 조절
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    
    # 증강이 없는 검증/테스트용 Transform 정의
    val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # 훈련용과 검증용 데이터셋을 각각 생성.
    train_dataset = EmotionDataset(data_dir=DATA_DIR / "train", transform=train_transform)
    val_dataset = EmotionDataset(data_dir=DATA_DIR / "val", transform=val_transform)

    # 데이터로더를 각각 생성. (검증용은 섞을 필요가 없음)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    NUM_CLASSES = len(train_dataset.classes)
    
    print("데이터 준비 완료!")
    print(f"훈련 데이터셋 크기: {len(train_dataset)}")
    print(f"클래스 수: {NUM_CLASSES} -> {train_dataset.classes}")

    # 모델, 손실 함수, 옵티마이저 준비
    model = create_model(model_name=MODEL_NAME, num_classes=NUM_CLASSES)
    # 모델을 지정된 장치로 이동
    model.to(DEVICE)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(
        model.parameters(), 
        #weight_decay=1e-4, #과적합 방지를 위한 가중치 감쇠를 넣었으나 오히려 학습에 방해가 되고 있음.
        lr=LEARNING_RATE 
        ) 
    scheduler = StepLR(optimizer, step_size=7, gamma=0.1)
    
    

    print(f"'{MODEL_NAME}' 모델, 손실 함수, 옵티마이저 준비 완료!")

    # 모델 훈련 시작
    print("\n모델 훈련을 시작합니다...")
    trained_model = train_model(model, 
                                train_loader, 
                                val_loader, 
                                criterion, 
                                optimizer, 
                                scheduler,
                                DEVICE, 
                                num_epochs=NUM_EPOCHS, 
                                patience=EARLY_STOPPING_PATIENCE)

    # 훈련된 모델 저장 (옵션)
    # torch.save(trained_model.state_dict(), f'{MODEL_NAME}_trained.pth')
    # print("훈련된 모델 가중치가 저장되었습니다.")